In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [19]:
import pywt

In [2]:
ts1 = np.loadtxt("ts1.txt")
ts2 = np.loadtxt("ts2.txt")
ts3 = np.loadtxt("ts3.txt")

In [28]:
def reconstruct_detail(coeffs, w_name, level):
    """
    Reconstruct detail coefficients at a specific level.

    Parameters:
    -----------
    coeffs : list
        Wavelet coefficients from wavedec.
    w_name : str
        Wavelet name.
    level : int
        The level to reconstruct.

    Returns:
    --------
    array_like
        Reconstructed detail coefficients.
    """
    coeffs_d = [np.zeros_like(c) for c in coeffs]
    coeffs_d[level] = coeffs[level]
    return pywt.waverec(coeffs_d, w_name)

In [62]:
coeffs = pywt.wavedec(ts1, 'db3', level=2)
det = pywt.upcoef('d', coeffs=coeffs[1], wavelet='db3', level=2, take=len(ts1)) # details reconstruction at level of decomp. 
x = np.sort(np.abs(det))
print(int(np.floor(len(x) * 0.98)))
#v2p100 = x[int(np.floor(len(x) * 0.98))]

980


In [56]:
v2p100

0.032815588962967074

In [3]:
import numpy as np
from Operations.CO_FirstCrossing import CO_FirstCrossing

def CO_glscf(y, alpha, beta, tau = 'tau'):
    """
    """
    # Set tau to first zero-crossing of the autocorrelation function with the input 'tau'
    if tau == 'tau':
        tau = CO_FirstCrossing(y, 'ac', 0, 'discrete')
    
    # Take magnitudes of time-delayed versions of the time series
    y1 = np.abs(y[:-tau])
    y2 = np.abs(y[tau:])


    p1 = np.mean(np.multiply((y1 ** alpha), (y2 ** beta)))
    p2 = np.multiply(np.mean(y1 ** alpha), np.mean(y2 ** beta))
    p3 = np.sqrt(np.mean(y1 ** (2*alpha)) - (np.mean(y1 ** alpha))**2)
    p4 = np.sqrt(np.mean(y2 ** (2*beta)) - (np.mean(y2 ** beta))**2)

    glscf = (p1 - p2) / (p3 * p4)

    return glscf    


In [11]:
CO_glscf(ts1, 0.9, 0.4, 3)

0.25284356429315435

In [13]:
import numpy as np
from Operations.CO_FirstCrossing import CO_FirstCrossing
from Operations.CO_glscf import CO_glscf

def CO_fzcglscf(y, alpha, beta, maxtau = None):
    """
    """
    N = len(y) # the length of the time series

    if maxtau is None:
        maxtau = N
    
    glscfs = np.zeros(maxtau)

    for i in range(1, maxtau+1):
        tau = i

        glscfs[i-1] = CO_glscf(y, alpha, beta, tau)
        if (i > 1) and (glscfs[i-1]*glscfs[i-2] < 0):
            # Draw a straight line between these two and look at where it hits zero
            out = i - 1 + glscfs[i-1]/(glscfs[i-1]-glscfs[i-2])
            return out
    
    return maxtau # if the function hasn't exited yet, set output to maxtau 


In [86]:
import warnings
import numpy as np

def DN_cv(x, k = 1):
    """
    Coefficient of variation

    Coefficient of variation of order k is sigma^k / mu^k (for sigma, standard
    deviation and mu, mean) of a data vector, x

    Parameters:
    ----------
    x (array-like): The input data vector
    k (int, optional): The order of coefficient of variation (k = 1 is default)

    Returns:
    -------
    float: The coefficient of variation of order k
    """
    if not isinstance(k, int) or k < 0:
        warnings.warn('k should probably be a positive integer')
        # Carry on with just this warning, though
    
    # Compute the coefficient of variation (of order k) of the data
    return (np.std(x, ddof=1) ** k) / (np.mean(x) ** k)


In [88]:
DN_cv(ts3)

26.269342687595913

In [93]:
import numpy as np

def DN_Quantile(y, p=0.5):
    """
    Calculates the quantile value at a specified proportion, p.

    Parameters:
    y (array-like): The input data vector
    p (float): The quantile proportion (default is 0.5, which is the median)

    Returns:
    float: The calculated quantile value

    Raises:
    ValueError: If p is not a number between 0 and 1
    """
    if p == 0.5:
        print("Using quantile p = 0.5 (median) by default")
    
    if not isinstance(p, (int, float)) or p < 0 or p > 1:
        raise ValueError("p must specify a proportion, in (0,1)")
    
    return np.quantile(y, p, method='hazen')


In [112]:
import numpy as np

def DN_nlogL_norm(y):
    """
    Negative log likelihood of data coming from a Gaussian distribution.

    This function fits a Gaussian distribution to the data and returns the negative
    log likelihood of the data coming from that Gaussian distribution.

    Parameters:
    y (array-like): A vector of data

    Returns:
    float: The negative log likelihood per data point
    """
    # Convert input to numpy array
    y = np.asarray(y)

    # Fit a Gaussian distribution to the data (mimicking MATLAB's normfit)
    mu = np.mean(y)
    sigma = np.std(y, ddof=1)  # ddof=1 for sample standard deviation

    # Compute the negative log-likelihood (mimicking MATLAB's normlike)
    n = len(y)
    nlogL = (n/2) * np.log(2*np.pi) + n*np.log(sigma) + np.sum((y - mu)**2) / (2*sigma**2)

    # Return the average negative log-likelihood
    return nlogL / n


In [113]:
DN_nlogL_norm(ts2)

1.2765413621752804

In [118]:
import numpy as np 

def DN_HighLowMu(y):
    """
    The highlowmu statistic.

    The highlowmu statistic is the ratio of the mean of the data that is above the
    (global) mean compared to the mean of the data that is below the global mean.

    Paramters:
    ----------
    y (array-like): The input data vector

    Returns:
    --------
    float: The highlowmu statistic.
    """
    mu = np.mean(y) # mean of data
    mhi = np.mean(y[y > mu]) # mean of data above the mean
    mlo = np.mean(y[y < mu]) # mean of data below the mean
    out = np.divide((mhi-mu), (mu-mlo)) # ratio of the differences

    return out


In [119]:
DN_HighLowMu(ts2)

1.0325203252032518